In [18]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import sys, re, os
import numpy as np
import pandas as pd
import glob,imp
from pandas import json_normalize
import pyslha
sys.path.append('/home/yoxara/smodels')
from smodels.particlesLoader import getParticlesFromSLHA
pd.options.mode.chained_assignment = None #Disable copy warnings
from smodels.base.model import Model
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.base.physicsUnits import fb, GeV
#from smodels.share.models.ppZpjj import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.smodelsLogging import setLogLevel
from smodels.experiment.defaultFinalStates import finalStates
setLogLevel("info")
#runtime.modelFile = 'smodels.share.models.ppZpjj'
!runSModelS.py -V

3.0.0-beta


In [19]:
BSMlist = getParticlesFromSLHA('TRV1jj_slha/run_01_mzp_1200_gb_0.01__.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
print(BSMlist)

[y1]


In [20]:
slhaFolder = 'TRV1jj_slha'
resultsFolder = 'TRV1jj_results'
File = "parameters.ini"
!chmod -R +rwx {slhaFolder}

In [21]:
!runSModelS.py -p {File} -f {slhaFolder} -o {resultsFolder}

INFO in databaseObj.loadTextDatabase() in 479: Parsing text database at /home/yoxara/smodels-database/
INFO in databaseObj.createBinaryFile() in 630: /home/yoxara/smodels-database/db3.pcl created.
INFO in modelTester.loadDatabaseResults() in 497: Including non-validated results
INFO in modelTester.testPoints() in 357: Running SModelS for 440 files with a single process. Messages will be redirected to smodels.log


In [22]:
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(resultsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)
    data.append(dataDict)

In [23]:
print(len(data))

440


In [24]:
smodelsDF = json_normalize(data)
smodelsDF

,filename,SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_31_mzp_1800_gb_0.01__.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",2.065917e+03,0.000000e+00,6.198799e+03,0.000000e+00,0.000000e+00,5.000000e-01,2.000000e-01,1,slhaFiles_Lept_gqp/run_01_mzp_1200_gb_0.01__.slha,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,slhaFiles_Lept_gqp/run_31_mzp_1800_gb_0.01__.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,4.132881e+03,2.333330e+02,1.062830e+02,[TRV1qq],"[(y1, 1800.0)]",CMS-EXO-12-059,None,8.000000e+00,1.970000e+01,upperLimit,1.771237e+01,3.888563e+01,"[(y1, 1.5278e-06)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,run_193_mzp_5000_gb_0.05__.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",4.640987e+02,0.000000e+00,1.392300e+03,0.000000e+00,0.000000e+00,5.000000e-01,2.000000e-01,1,slhaFiles_Lept_gqp/run_01_mzp_1200_gb_0.01__.slha,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,slhaFiles_Lept_gqp/run_193_mzp_5000_gb_0.05__....,3.0.0-beta,3.0.0-beta,0.000000e+00,9.282013e+02,1.116670e+00,1.836310e+00,[TRV1qq],"[(y1, 5000.0)]",CMS-EXO-12-059,None,8.000000e+00,1.970000e+01,upperLimit,8.312226e+02,5.054709e+02,"[(y1, 0.00016579)]"


In [25]:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 5000001])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['DMINPUTS'].items()])
    #extparDict.update(dict([[str(key),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 9000006])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)

In [26]:
#Convert to DataFrame
pd.set_option('display.float_format', '{:e}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 2)
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [27]:
dataDF = slhaDF.merge(smodelsDF,how='inner')

In [28]:
print('Final number of data points:',dataDF.shape[0])


Final number of data points: 440


In [29]:
dataDF.to_csv('TRV1jj_data.csv')
print(len(dataDF))
dataDF

440


,filename,mass.5000001,extpar.1,xsec8TeV(fb).5000001,"BRs.y1.b,b","BRs.y1.c,c","BRs.y1.q,q","BRs.y1.t-,t+",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_31_mzp_1800_gb_0.01__.slha,1.800000e+03,8.000000e-03,6.198800e+03,1.666807e-01,1.666807e-01,1.666807e-01,1.665963e-01,0.000000e+00,6.198800e+03,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",2.065917e+03,0.000000e+00,6.198799e+03,0.000000e+00,0.000000e+00,5.000000e-01,2.000000e-01,1,slhaFiles_Lept_gqp/run_01_mzp_1200_gb_0.01__.slha,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,slhaFiles_Lept_gqp/run_31_mzp_1800_gb_0.01__.slha,3.0.0-beta,3.0.0-beta,0.000000e+00,4.132881e+03,2.333330e+02,1.062830e+02,[TRV1qq],"[(y1, 1800.0)]",CMS-EXO-12-059,None,8.000000e+00,1.970000e+01,upperLimit,1.771237e+01,3.888563e+01,"[(y1, 1.5278e-06)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,run_193_mzp_5000_gb_0.05__.slha,5.000000e+03,5.000000e-02,1.392300e+03,1.666669e-01,1.666669e-01,1.666669e-01,1.666655e-01,0.000000e+00,1.392300e+03,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",4.640987e+02,0.000000e+00,1.392300e+03,0.000000e+00,0.000000e+00,5.000000e-01,2.000000e-01,1,slhaFiles_Lept_gqp/run_01_mzp_1200_gb_0.01__.slha,1.000000e-11,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,slhaFiles_Lept_gqp/run_193_mzp_5000_gb_0.05__....,3.0.0-beta,3.0.0-beta,0.000000e+00,9.282013e+02,1.116670e+00,1.836310e+00,[TRV1qq],"[(y1, 5000.0)]",CMS-EXO-12-059,None,8.000000e+00,1.970000e+01,upperLimit,8.312226e+02,5.054709e+02,"[(y1, 0.00016579)]"


In [30]:
dataDF['xsec8TeV(fb).5000001'] * (3*dataDF["BRs.y1.q,q"] + dataDF["BRs.y1.c,c"] )

0     4.132881e+03
          ...     
439   9.282013e+02
Length: 440, dtype: float64

In [31]:
dataDF['ExptRes.result0.theory prediction (fb)']

0     4.132881e+03
          ...     
439   9.282013e+02
Name: ExptRes.result0.theory prediction (fb), Length: 440, dtype: float64

In [32]:
(dataDF['xsec8TeV(fb).5000001'] * (3*dataDF["BRs.y1.q,q"] + dataDF["BRs.y1.c,c"] )) /dataDF['ExptRes.result0.theory prediction (fb)']

0     1.000000e+00
          ...     
439   1.000000e+00
Length: 440, dtype: float64

In [33]:
dataDF['ExptRes.result0.r']

0     1.771237e+01
          ...     
439   8.312226e+02
Name: ExptRes.result0.r, Length: 440, dtype: float64

In [34]:
mask = dataDF['ExptRes.result0.r'].isna()
rows_with_nan = dataDF[mask]
rows_with_nan.to_csv('rows_with_nan_.csv', index=False)
rows_with_nan

,filename,mass.5000001,extpar.1,xsec8TeV(fb).5000001,"BRs.y1.b,b","BRs.y1.c,c","BRs.y1.q,q","BRs.y1.t-,t+",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
